In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools 
import multiprocessing as mp
import numpy as np
import sys
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score

sys.path.append('../')
from baseModels.SVM.model import SVM

In [2]:
feature_dir = '../featEngg/offline/kernelMethods/data/featvec/trainfiles/'
feature_files = [f.name for f in os.scandir(feature_dir) if f.name.endswith('.csv.gz')]

In [3]:
label_file = '../data/label/EnzymeLabelsMultiClass.csv'

In [4]:
# hyperparameter optimization file
hyperparam_file = '../data/results/HyperParameterOptimization/IndHPOpt.csv'

df_hyperparam = pd.read_csv(hyperparam_file).set_index('feat_name')

In [5]:
# append model results to file
filename_model = '../data/results/feat_results.csv'

In [6]:
def get_featname_from_file(filename):
    featname = filename.replace('.csv.gz','')
    return featname

In [7]:
def model_evaluate(filename,random_seed):
    featname = get_featname_from_file(filename)
    
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    
    pca_components = df_hyperparam.loc[featname,'pca_comp']
    regCparam = df_hyperparam.loc[featname,'regC']
    kernparam = df_hyperparam.loc[featname,'kernel']


    X_train, X_valid, y_train, y_valid,enz_train,enz_valid = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    
    if X_train.shape[1]<pca_components:
        raise ValueError('Wrong Hyper Parameter')
        
    svm = SVM(X_train,X_valid,y_train,y_valid,verbose=False,optimize=False, pca_comp=pca_components,kern=kernparam,regC=regCparam)

    yi = svm.yvalid
    yhati = svm.ypredvalid
    prec=get_precision(yi,yhati)
    rec=get_recall(yi,yhati)
    f1_score =get_f1_score(yi,yhati)
    acc =get_accuracy(yi,yhati)
    
    return featname,prec,rec,f1_score,acc


def get_precision(y,yhat):
    return round(precision_score(y,yhat,labels=[3],average='micro'),2)


def get_recall(y,yhat):
    return round(recall_score(y,yhat,labels=[3],average='micro'),2)


def get_f1_score(y,yhat):
    return round(f1_score(y,yhat,average='micro'),2)

def get_accuracy(y,yhat):
    return round(accuracy_score(y,yhat),2)

def write_model_stats(model_metrics):
    datadict = {'precision':[],'recall':[],'f1_score':[],'accuracy':[]}
    index_names = []
    
    for name,group in itertools.groupby(model_metrics,lambda x: x[0]):
        
        index_names.append(name)
        metrics = list(group)
        precs = [g[1] for g in metrics]
        recalls = [g[2] for g in metrics]
        f1_scores = [g[3] for g in metrics]
        accs = [g[4] for g in metrics]
            
        datadict['precision'].append(np.mean(precs))
        datadict['recall'].append(np.mean(recalls))
        datadict['f1_score'].append(np.mean(f1_scores))
        datadict['accuracy'].append(np.mean(accs))

    df = pd.DataFrame(datadict,index=index_names)
    return df.to_csv(filename_model,mode='a',index=True,header=False)       


In [8]:
mp.cpu_count()

24

In [9]:
pool = mp.Pool(mp.cpu_count())

In [11]:
iterable = list(itertools.product(feature_files,range(10000)))

In [12]:
%%time
models = list(pool.starmap(model_evaluate,iterable))

CPU times: user 413 ms, sys: 94.4 ms, total: 508 ms
Wall time: 29min 21s


In [14]:
write_model_stats(models)

# Previous Results Generation - No longer necessary

In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv.gz','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('mismatchKernel')

{'train_min': 0.9186046511627907,
 'train_mean': 0.9638430232558142,
 'train_max': 1.0,
 'train_std': 0.012560883367346839,
 'test_min': 0.4482758620689655,
 'test_mean': 0.7745793103448275,
 'test_max': 1.0,
 'test_std': 0.07275956985816574}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('../data/SimResults/KernelResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
mismatchKernel,0.918605,0.963843,1.0,0.012561,0.448276,0.774579,1.0,0.072760
gappyKernel,0.941860,0.974081,1.0,0.011225,0.413793,0.769362,1.0,0.075005
spectrumKernel,0.930233,0.974930,1.0,0.011244,0.413793,0.766100,1.0,0.075272
